In [2]:
pip install xgboost

     |████████████████████████████████| 255.9 MB 29 kB/s /s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import shutil
from joblib import dump, load
import pandas as pd

import xgboost as xgb

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

import mlflow
import argparse

In [11]:
df = pd.read_csv("head_processed_data.csv")

In [16]:
df.head()

,Unnamed: 0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
X = df.loc[:, ~df.columns.isin(['Unnamed: 0', 'SK_ID_CURR', 'TARGET'])]

In [17]:
y = df[['TARGET']]

In [19]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
pip install hyperopt


     |████████████████████████████████| 1.6 MB 31.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 102.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 1.1 MB/s s eta 0:00:01
     |████████████████████████████████| 829 kB 109.8 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=fb02bea1798a12429bf27f2170afdf890efeeb3f25d282d37b6f3892861bea3d
  Stored in directory: /home/cdsw/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future


In [21]:
import time
import warnings
import numpy as np
#hyperparameter tuning
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK
from hyperopt.pyll import scope
from sklearn.metrics import roc_auc_score

In [22]:
df.head()

,Unnamed: 0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.count()

Unnamed: 0                          9
SK_ID_CURR                          9
TARGET                              9
CODE_GENDER                         9
FLAG_OWN_CAR                        9
                                   ..
CC_NAME_CONTRACT_STATUS_nan_MAX     2
CC_NAME_CONTRACT_STATUS_nan_MEAN    2
CC_NAME_CONTRACT_STATUS_nan_SUM     2
CC_NAME_CONTRACT_STATUS_nan_VAR     2
CC_COUNT                            2
Length: 721, dtype: int64

In [24]:
default_params = {}
xgbclf = xgb.XGBClassifier()
gparams = xgbclf.get_params()

#default parameters have to be wrapped in lists - even single values - so GridSearchCV can take them as inputs
for key in gparams.keys():
    gp = gparams[key]
    default_params[key] = [gp]

# Create XGBoost DMatrix objects for efficient data handling

train = xgb.DMatrix(data=X_train, label=y_train)
test = xgb.DMatrix(data=X_test, label=y_test)
    
#list of hyperparameters available to Tune
default_params

{'objective': ['binary:logistic'],
 'use_label_encoder': [False],
 'base_score': [None],
 'booster': [None],
 'callbacks': [None],
 'colsample_bylevel': [None],
 'colsample_bynode': [None],
 'colsample_bytree': [None],
 'early_stopping_rounds': [None],
 'enable_categorical': [False],
 'eval_metric': [None],
 'gamma': [None],
 'gpu_id': [None],
 'grow_policy': [None],
 'importance_type': [None],
 'interaction_constraints': [None],
 'learning_rate': [None],
 'max_bin': [None],
 'max_cat_to_onehot': [None],
 'max_delta_step': [None],
 'max_depth': [None],
 'max_leaves': [None],
 'min_child_weight': [None],
 'missing': [nan],
 'monotone_constraints': [None],
 'n_estimators': [100],
 'n_jobs': [None],
 'num_parallel_tree': [None],
 'predictor': [None],
 'random_state': [None],
 'reg_alpha': [None],
 'reg_lambda': [None],
 'sampling_method': [None],
 'scale_pos_weight': [None],
 'subsample': [None],
 'tree_method': [None],
 'validate_parameters': [None],
 'verbosity': [None]}

In [25]:
search_space = {
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 100)),
    'min_child_weight': hp.loguniform('min_child_weight', -2, 3),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'gamma': hp.loguniform('gamma', -10, 10),
    'alpha': hp.loguniform('alpha', -10, 10),
    'lambda': hp.loguniform('lambda', -10, 10),
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'seed': 123,
}

In [ ]:
# With MLflow autologging, hyperparameters and the trained model are automatically logged to MLflow.
def train_model(params):
    mlflow.xgboost.autolog(silent=True)

    # However, we can log additional information by using an MLFlow tracking context manager
    with mlflow.start_run(nested=True):

        # Train model and record run time
        start_time = time.time()
        booster = xgb.train(params=params, dtrain=train, num_boost_round=5000, evals=[(test, "test")], early_stopping_rounds=50, verbose_eval=False)
        run_time = time.time() - start_time
        mlflow.log_metric('runtime', run_time)

        # Record AUC as primary loss for Hyperopt to minimize
        predictions_test = booster.predict(test)
        auc_score = roc_auc_score(y_test, predictions_test)
        mlflow.log_metric('test-auc', auc_score)

        # Set the loss to -1*auc_score so fmin maximizes the auc_score
        return {'status': STATUS_OK, 'loss': -auc_score, 'booster': booster.attributes()}